# **IME Data Analytics Competition**

> For this competition, I used Python to optimize the monetary gains from a stock market investment of $1500 based on stock price data provided from the S&P 500 from 2/8/2013 to 2/7/2018. I used Google Colab as my IDE because of its free access to GPUs, which would come to be beneficial when computing the optimizations





### Step 0: Packages and Libraries


> I chose to use the PuLP library, which includes a linear programming model that can be utilized for optimizing our objective function. Additionally, I used the pandas library because it provides functionality on how to work with dataframes, such as the csv dataframe that was provided. 

> After importing and downloading everything I need, I connected to a GPU offered by Google Colab for free so that I could run my intensive computations.








In [2]:
!pip install pulp

     |████████████████████████████████| 40.6MB 72kB/s 


In [0]:
import pandas as pd
import io
from pulp import *
from dateutil.relativedelta import relativedelta
from google.colab import files
import matplotlib as plt

In [4]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [5]:
!nvidia-smi

Tue May 26 11:08:15 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

### Step 1: Data Cleaning/Alteration

> I import and read the CSV that we were given and noticed that we only had the date, code, and price. It was difficult to gather the overall trend of the stocks because they were not organized, so I created a pivot table. Now, it is easier to see all the prices of every stock on a given day. 



In [6]:
uploaded = files.upload()

Saving DAComp2Data.csv to DAComp2Data.csv


In [7]:
stocks = pd.read_csv('DAComp2Data.csv')
stocks.info()
stocks

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493528 entries, 0 to 493527
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Date    493528 non-null  object 
 1   Price   493528 non-null  float64
 2   Code    493528 non-null  object 
dtypes: float64(1), object(2)
memory usage: 11.3+ MB


,Date,Price,Code
0,5/19/16,39.82,HRUA
1,5/20/16,40.29,HRUA
2,5/23/16,40.45,HRUA
3,5/24/16,41.13,HRUA
4,5/25/16,41.29,HRUA
...,...,...,...
493523,2/1/18,23.57,ZYSG
493524,2/2/18,23.86,ZYSG
493525,2/5/18,23.25,ZYSG
493526,2/6/18,23.46,ZYSG


In [8]:
def pivot_maker(csvname):
  df = pd.read_csv(io.BytesIO(uploaded[csvname]))
  df['Date'] = pd.to_datetime(df['Date']).dt.date
  pivot = pd.pivot_table(df, values = 'Price', index = 'Date', columns = 'Code')
  return pivot

piv = pivot_maker('DAComp2Data.csv')
piv.info()
piv

<class 'pandas.core.frame.DataFrame'>
Index: 1259 entries, 2013-02-08 to 2018-02-07
Columns: 392 entries, ACFV to ZYSG
dtypes: float64(392)
memory usage: 3.8+ MB


Code,ACFV,ACXN,AEMS,AENO,AGTX,AGTZ,AHGP,AIDX,AIHG,AJCH,AJSI,ALVV,ANXW,ANZJ,ARFC,ASVI,ASVW,AXEP,AZOQ,BAQO,BFTH,BGTD,BHGB,BHTN,BINW,BJLF,BKRD,BLNJ,BMDX,BNTO,BROR,BTBY,BVOP,BWJZ,BWKW,BYSX,CALD,CDBX,CFKR,CFPE,...,WQFA,WQWS,WSWV,WZDT,WZTS,XBGS,XCQX,XDLL,XHWQ,XIEB,XLHR,XRCT,XTYY,XWCE,XXBK,YCKU,YDLE,YFPZ,YGAL,YGJY,YGRZ,YKGD,YLLB,YNVW,YPHK,YSNS,YSOM,YTXK,YVTE,ZCKL,ZESE,ZIVA,ZJNC,ZJSJ,ZKBW,ZLLA,ZOQI,ZPZG,ZRBQ,ZYSG
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2013-02-08,42.83,11.07,24.40,52.57,21.97,151.86,37.60,45.73,101.15,56.53,57.74,34.88,216.71,44.35,41.46,24.335,34.53,31.84,15.29,36.25,46.01,69.00,16.81,59.28,47.10,261.95,52.70,151.60,21.16,164.44,50.60,119.47,69.08,42.77,62.91,45.39,72.91,57.87,32.73,88.61,...,80.89,43.610,90.90,50.690,71.13,24.06,37.64,32.2094,39.18,103.11,28.42,27.45,63.78,27.84,48.93,53.09,74.40,40.22,92.53,93.66,62.49,32.61,34.460,53.95,30.065,10.1275,57.60,69.490,30.65,27.31,98.27,47.50,26.50,65.40,38.79,22.50,60.49,25.66,24.13,46.70
2013-02-11,42.28,11.20,24.45,52.13,22.65,151.86,37.65,45.42,102.12,56.66,57.12,35.26,217.61,44.32,41.51,24.400,34.31,31.96,15.71,35.85,46.11,69.14,16.90,59.31,47.22,257.21,52.69,152.24,21.27,163.96,50.38,119.26,68.26,42.69,62.74,45.66,72.40,57.59,33.03,88.28,...,79.31,43.525,90.40,50.740,71.06,24.17,37.11,32.1780,39.04,104.84,28.19,27.33,63.90,27.94,48.93,53.32,74.06,40.25,92.47,92.30,62.67,32.84,34.780,54.13,29.815,10.0975,57.21,69.280,30.63,27.43,98.07,47.79,26.48,65.21,39.45,22.45,60.22,25.67,24.12,46.91
2013-02-12,43.21,11.31,24.74,52.50,22.64,152.65,37.74,45.32,101.77,56.70,57.01,35.51,223.09,44.44,41.82,24.430,34.61,31.84,15.43,35.42,46.06,69.50,17.18,59.12,47.11,258.70,52.93,154.11,20.97,165.45,50.15,119.19,68.92,42.72,62.49,45.53,73.59,58.25,33.02,88.46,...,80.00,43.710,89.90,51.000,71.46,24.17,37.66,32.4712,39.22,106.38,28.61,27.35,64.01,28.00,49.01,54.30,74.46,40.50,91.99,90.21,62.73,33.15,34.960,54.49,29.990,10.0350,57.00,68.271,30.60,27.71,98.43,47.93,26.69,65.69,38.63,22.58,59.49,25.84,24.24,47.08
2013-02-13,42.67,11.34,24.84,52.85,22.87,152.85,37.80,42.69,102.81,57.28,57.23,35.13,225.62,44.52,42.00,24.760,34.96,32.00,15.12,35.27,46.60,68.82,17.15,59.55,46.99,269.47,53.22,154.52,21.14,165.91,49.51,119.04,68.96,42.61,62.50,45.58,72.39,58.61,32.75,88.67,...,80.03,43.530,89.89,52.005,71.61,24.49,37.28,32.9843,39.41,107.57,28.99,27.73,63.88,27.92,48.77,55.49,74.67,40.68,93.22,92.01,62.69,33.28,34.990,54.56,29.810,10.1025,57.33,67.450,30.52,27.63,99.53,48.04,27.00,65.50,38.87,23.39,59.14,25.86,24.12,47.26
2013-02-14,42.94,11.21,24.86,53.63,22.93,152.49,37.57,42.80,102.00,57.30,57.05,35.21,226.29,44.33,41.91,24.355,35.34,32.12,15.71,36.57,46.98,68.00,17.15,59.95,47.17,269.24,53.09,155.93,20.99,164.59,49.56,118.73,70.29,42.43,62.66,45.67,72.01,57.92,32.56,88.52,...,79.48,43.175,90.19,52.490,71.86,24.40,36.84,33.3717,39.15,107.05,29.11,27.48,64.02,27.89,48.20,55.90,74.82,40.49,93.47,87.63,63.09,33.14,35.690,54.33,29.640,10.1050,57.06,68.110,30.36,27.78,100.35,48.17,26.70,65.37,39.21,23.41,59.13,25.75,23.89,47.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-02-01,58.93,11.51,26.15,97.00,56.58,227.17,18.06,120.10,121.89,144.48,235.22,65.51,266.13,54.30,77.91,64.080,18.43,32.75,73.10,116.34,65.09,76.84,20.57,33.21,84.98,1390.00,106.78,272.23,41.70,346.00,187.39,189.17,37.43,42.57,52.64,114.70,93.43,59.35,255.57,89.07,...,371.18,126.980,117.90,54.500,105.34,53.28,47.16,39.8100,81.84,276.08,38.26,26.20,164.78,44.81,46.96,179.65,138.49,32.62,217.37,118.09,74.97,55.21,136.560,78.46,48.490,21.4500,120.52,174.730,55.70,36.03,224.08,74.72,51.42,107.98,64.17,16.02,139.41,44.16,26.43,23.57
2018-02-02,55.40,11.12,25.46,96.75,55.25,225.38,17.50,118.71,121.00,143.59

### Step 2: Creating Optimization Functions

> In order to solve the problem, I first considered a nolinear optimization problem with our objective function as the sell price minus the buy price. I realized that iterating through and solving that problem for every possible combination of dates would give me the maximum possible earnings.


In [0]:
''' 
This is the definition for the optimization function based on the specific sell and buy day. Using the PuLP package, we are able to define the objective
function and constraints. This is essentially excel solver, but in python. If details = False, it returns the sell date and the maximum earnings.
This is what we will set it to when using it in the the full_optimization function to find the max earnings. After we find out which days produce the highest
earnings, we can set details = True which will return the stock names and how many shares of each we bought.
'''
def optimization(data, buy_row_index, sell_row_index, stock_names, details = False):
  prob = LpProblem('MAXRETURNS', LpMaximize)  # Find Maximum
            
  sellprice = dict(zip(stock_names, data.iloc[sell_row_index].values))
  buyprice = dict(zip(stock_names, data.iloc[buy_row_index].values))

  IntShares = LpVariable.dicts('IntShares', stock_names, lowBound=0, upBound=10, cat='Integer')  # Initialize integer variables, constrain to 10 maximum shares
  BinTicker = LpVariable.dicts('BinTicker', stock_names, cat='Binary')  # Initialize binary variables
  prob += lpSum(IntShares[i] * (sellprice[i] - buyprice[i]) for i in stock_names)  #Objective Function
            
  #Subject To:
  prob += lpSum([IntShares[i] * buyprice[i] for i in stock_names]) <= 1500, 'Budget (1500)'
  prob += lpSum([BinTicker[i] for i in stock_names]) >= 5, 'Min Tickers (5)'
  for i in stock_names:
    prob += (IntShares[i] - ((10 * BinTicker[i]))) <= 0
    prob += (IntShares[i] - (.5 * BinTicker[i])) >= 0

  prob.solve()

  if details == True:
    details = []
    for v in prob.variables():
      if v.varValue > 0:
          details.append({v.name: v.varValue})
    details.append({'Maximum Earnings': value(prob.objective)})
    return details

  print(str(data.index[sell_row_index]), value(prob.objective))
  return (str(data.index[sell_row_index]), value(prob.objective))

In [0]:
''' 
We want to identify a buy date and then iterate through all the possible sell dates for that buy date (within a year) and calculate the maximum earnings of that given pair
We then want to store the largest maximum earnings and the associated sell date that we got from the buy date
We repeat this process for every buy date until we have stored all pairings of the buy dates and best sell dates 
Example: We will have something like (buydate1, best selldate for 1): max_earnings from that date pairing
                                     (buydate2, best selldate for 2): max_earnings from that date pairing
                                     (buydate3, best selldate for 3): max_earnings from that date pairing
                                                              ....
                                     (buydate1258, best selldate for 1258): max_earnings from that date pairing
                                     (buydate1259, best selldate for 1259): max_earnings from that date pairing
'''

def full_optimization(data, start, stop):
  max_earnings_buyday = {}
  labels = data.columns
  
  #Larger loop that iterates through all buy dates
  for buyidx in range(start, stop):
    buydate = data.index[buyidx]
    buy_prices = data.iloc[buyidx].values

    max_price = 0
    sell_day = ""

    #Smaller loop that iterates through all possible sell dates given the buy day
    for sellidx in range(buyidx + 1, piv.shape[0]):

      selldate = data.index[sellidx]

      less_than_a_year = relativedelta(selldate, buydate).years
      if less_than_a_year < 1:
        top_earning_details = optimization(data, buyidx, sellidx, labels)

        if max_price < top_earning_details[1]:
          max_price = top_earning_details[1]
          sell_day = top_earning_details[0]
       
      #Checking special case where buy and sell date are a year apart, structured code this way to only check .days when necessary
      else: 
        exactly_one_year = relativedelta(selldate, buydate).years + relativedelta(selldate, buydate).days
        if exactly_one_year == 1:
          top_earning_details = optimization(data, buyidx, sellidx, labels)

          if max_price < top_earning_details[1]:
            max_price = top_earning_details[1]
            sell_day = top_earning_details[0]
        break

    max_earnings_buyday[str(data.index[buyidx]) + " TO " + sell_day] = max_price

  return max_earnings_buyday

### Step 3: Gathering Optimization Data

> Because it takes approximately 40 seconds to find the sell date and maximum earnings for a given buy date, the code will run for around 14 hours. In order to reduce the time cost, I divided the data in four parts and ran the optimizations on 4 different Google Colab accounts in parallel so that the time for all the notebooks to finish running would be around 3.5 hours, preventing Google Colab from timing out. I then combined all the four individual csv instances into one and was able to look at all the dates and their statistics.


In [0]:
# max1 = full_optimization(piv, 0, 315)
# max_section_1 = pd.DataFrame(list(max1.items()), columns=['Dates','Max Earnings'])
# max_section_1.to_csv('max1real.csv')
# !cp max1real.csv "drive/My Drive/"


# max2 = full_optimization(piv, 315, 630)
# max_section_2 = pd.DataFrame(list(max2.items()), columns=['Dates','Max Earnings'])
# max_section_2.to_csv('max2.csv')
# !cp max2.csv "drive/My Drive/"


# max3 = full_optimization(piv, 630, 945)
# max_section_3 = pd.DataFrame(list(max3.items()), columns=['Dates','Max Earnings'])
# max_section_3.to_csv('max3.csv')
# !cp max3.csv "drive/My Drive/"


# max4 = full_optimization(piv, 945, 1259)
# max_section_4 = pd.DataFrame(list(max4.items()), columns=['Dates','Max Earnings'])
# max_section_4.to_csv('max4.csv')
# !cp max2.csv "drive/My Drive/"

In [0]:
m_all = files.upload()

Saving maxcombined.csv to maxcombined.csv


In [0]:
dfm_combined = pd.read_csv(io.BytesIO(m_all['maxcombined.csv'])).drop('Unnamed: 0', axis = 1)
dfm_combined



,Dates,Max Earnings
0,2013-02-08 TO 2014-02-07,2643.584
1,2013-02-11 TO 2014-02-10,2728.101
2,2013-02-12 TO 2014-02-10,2735.970
3,2013-02-13 TO 2014-02-13,2736.944
4,2013-02-14 TO 2014-02-13,2725.528
...,...,...
1254,2018-02-01 TO 2018-02-07,147.670
1255,2018-02-02 TO 2018-02-07,166.540
1256,2018-02-05 TO 2018-02-07,186.880
1257,2018-02-06 TO 2018-02-07,128.140


### Step 4: Identifying Best Buy Date and Sell Date

> After combining all the four parts, I sorted the dataframe and identified the best buy and sell date. I found that the best buy date is Feb 25, 2013 and the best sell date is Feb 25, 2014 and the earnings I got was $3196.5725!



In [0]:
dfm_combined = dfm_combined.sort_values(by = "Max Earnings", ascending = False, ignore_index = True)
dfm_combined

,Dates,Max Earnings
0,2013-02-25 TO 2014-02-25,3196.5725
1,2013-02-26 TO 2014-02-25,3152.5700
2,2013-02-27 TO 2014-02-25,3096.9700
3,2016-02-11 TO 2017-02-03,3081.0500
4,2013-02-28 TO 2014-02-25,3059.3700
...,...,...
1254,2018-01-30 TO 2018-02-07,152.7700
1255,2018-01-29 TO 2018-02-01,148.0600
1256,2018-02-01 TO 2018-02-07,147.6700
1257,2018-02-06 TO 2018-02-07,128.1400


In [0]:
# Best buy day/sell day combination

dfm_combined.iloc[0]

Dates           2013-02-25 TO 2014-02-25
Max Earnings                     3196.57
Name: 0, dtype: object

### Step 5: Identifying distribution of stock purchases and shares

> Now that I know the dates, I need to identify what stocks and number of shares to purchase. I can call the optimization() function which takes in an additional argument known as details. Here, I can specify the start and end date and get back the distribution of stock purchases and shares.

> These are the results:


*   EMFA: 10 shares
*   KBZK: 10 shares
*   KZIM: 7 shares
*   PDRG: 1 share
*   RIRH: 10 shares
*   WKCS: 10 shares















In [0]:
# Indices of the buy and sell prices in the pivot table
buy_prices = piv.iloc[10]
sell_prices = piv.iloc[262]
optimization(piv, 10, 262, piv.columns, True)

[{'BinTicker_EFMA': 1.0},
 {'BinTicker_KBZK': 1.0},
 {'BinTicker_KZIM': 1.0},
 {'BinTicker_PDRG': 1.0},
 {'BinTicker_RIRH': 1.0},
 {'BinTicker_WKCS': 1.0},
 {'IntShares_EFMA': 10.0},
 {'IntShares_KBZK': 10.0},
 {'IntShares_KZIM': 7.0},
 {'IntShares_PDRG': 1.0},
 {'IntShares_RIRH': 10.0},
 {'IntShares_WKCS': 10.0},
 {'Maximum Earnings': 3196.5725}]

In [0]:
stocks = ['EFMA', 'KBZK', 'KZIM', 'PDRG', 'RIRH', 'WKCS']
shares = [10, 10, 7, 1, 10, 10]
budget_spent = 1500
money_back = 0
total_earnings = 0

print("STARTING BUDGET: " + str(budget_spent))
for s in range(len(stocks)):
  bprice = buy_prices[buy_prices.index == stocks[s]].values[0]
  sprice = sell_prices[sell_prices.index == stocks[s]].values[0]
  
  budget_spent -= bprice * shares[s]
  money_back += sprice * shares[s]
  total_earnings += (sprice - bprice) * shares[s]
  rev = sprice - bprice

  print(stocks[s] + ": " + str(shares[s]) + " shares")
  print()

print("TOTAL MONEY SPENT: " + str(1500 - budget_spent))
print("TOTAL MONEY BACK: " + str(money_back))
print("TOTAL EARNINGS: " + str(total_earnings))

STARTING BUDGET: 1500
EFMA: 10 shares

KBZK: 10 shares

KZIM: 7 shares

PDRG: 1 shares

RIRH: 10 shares

WKCS: 10 shares

TOTAL MONEY SPENT: 1499.2525
TOTAL MONEY BACK: 4695.825
TOTAL EARNINGS: 3196.5725


# Conclusion

In order to maximize the monetary gains from a stock market investment of $1500 based on stock price data provided from the S&P 500 from 2/8/2013 to 2/7/2018, I should buy and selling the following stocks and shares on February 25, 2013 and February 25, 2014.

*   EMFA: 10 shares
*   KBZK: 10 shares
*   KZIM: 7 shares
*   PDRG: 1 share
*   RIRH: 10 shares
*   WKCS: 10 shares

As a result, I will earn 3196.5725 dollars from the initial 1500 dollars budget

